# Predicción de Ventas de Vehículos

El propósito del proyecto es desarrollar un modelo predictivo para determinar la probabilidad de venta de vehículos en función de su kilometraje, edad del modelo y precio. 

Para ello, se llevarán a cabo pasos de tratamiento y procesamiento de datos, adaptando el dataset a las necesidades del proyecto. El modelo permitirá evaluar si un vehículo con determinadas características puede ser vendido al precio ofrecido o si es necesario ajustar el precio para facilitar la venta.

## Tratamiento de datos


### Recopilación de datos

El siguiente dataset contiene información sobre vehículos y sus características. Incluye las siguientes columnas:

- mileage_per_year: Kilometraje promedio por año del vehículo.
- model_year: Año del modelo del vehículo.
- price: Precio del vehículo.
- sold: Indica si el vehículo fue vendido ("yes") o no ("no").

In [1]:
import pandas as pd
uri = 'https://gist.githubusercontent.com/ahcamachod/1595316a6b37bf39baac355b081d9c3b/raw/98bc94de744764cef0e67922ddfac2a226ad6a6f/car_prices.csv'

datos = pd.read_csv(uri)
datos.head()

,Unnamed: 0,mileage_per_year,model_year,price,sold
0,0,21801,2000,30941.02,yes
1,1,7843,1998,40557.96,yes
2,2,7109,2006,89627.50,no
3,3,26823,2015,95276.14,no
4,4,7935,2014,117384.68,yes


### Transformación y limpieza de datos

In [2]:
mapa = {
        'mileage_per_year':'millas_por_ano',
        'model_year':'ano_del_modelo',
        'price':'precio',
        'sold':'vendido'
        }

datos = datos.rename(columns=mapa)
datos.sample(3)

,Unnamed: 0,millas_por_ano,ano_del_modelo,precio,vendido
7970,7970,17487,1999,56805.47,yes
4371,4371,13495,2001,38940.10,yes
9902,9902,11958,2001,79477.79,yes


In [3]:
cambio = {'no':0, 'yes':1}

datos.vendido = datos.vendido.map(cambio)
datos.sample(3)

,Unnamed: 0,millas_por_ano,ano_del_modelo,precio,vendido
6484,6484,10068,2006,43173.92,1
4790,4790,16138,2002,53795.20,1
6998,6998,9789,2000,30726.81,1


In [4]:
from datetime import datetime

ano_actual = datetime.today().year
datos['edad_del_modelo'] = ano_actual - datos.ano_del_modelo
datos.sample(3)

,Unnamed: 0,millas_por_ano,ano_del_modelo,precio,vendido,edad_del_modelo
9982,9982,10058,1999,58657.16,0,25
3500,3500,16988,2002,115665.50,1,22
1729,1729,11074,1999,68718.20,1,25


In [5]:
datos['km_por_ano'] = datos.millas_por_ano * 1.60934
datos.sample(3)

,Unnamed: 0,millas_por_ano,ano_del_modelo,precio,vendido,edad_del_modelo,km_por_ano
1931,1931,18460,2000,79119.09,0,24,29708.41640
7838,7838,11149,1998,109594.28,1,26,17942.53166
7069,7069,18116,1999,58033.24,1,25,29154.80344


In [6]:
datos = datos.drop(columns=['Unnamed: 0', 'millas_por_ano','ano_del_modelo'], axis=1)
datos.sample(3)

,precio,vendido,edad_del_modelo,km_por_ano
1837,76511.55,0,26,15287.12066
5943,89115.67,1,26,23317.72726
3599,63960.80,0,15,17348.68520


### División del Conjunto de Datos

In [7]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

x= datos[['edad_del_modelo','km_por_ano', 'precio']]
y= datos.vendido

SEED = 42
np.random.seed(SEED)

raw_x_train, raw_x_test, y_train, y_test = train_test_split(x,y,test_size=0.25,stratify=y)
print(f"Entrenaremos con {len(raw_x_train)} elementos y probaremos con {len(raw_x_test)} elementos.")


Entrenaremos con 7500 elementos y probaremos con 2500 elementos.


### Escalamiento de datos

In [8]:

scaler = StandardScaler()
scaler.fit(raw_x_train)
x_train = scaler.transform(raw_x_train)
x_test = scaler.transform(raw_x_test)


## Implementación del Modelo de Predicción

### Selección del Modelo

####  Support Vector Classifier (SVC)

Las SVM buscan encontrar el hiperplano que mejor separa las clases en el espacio de características, maximizando el margen entre las clases más cercanas de cada categoría. En este caso, se usa para predecir si un vehículo será vendido (yes) o no (no) basado en su kilometraje, edad del modelo y precio.

In [9]:
model = SVC()
model.fit(x_train,y_train)
previsiones= model.predict(x_test)

tasa_de_acierto = accuracy_score(y_test, previsiones)*100
print(f'La exactitud del clasificador SVM fue: {round(tasa_de_acierto,2)}%')

La exactitud del clasificador SVM fue: 76.04%


#### Decision Tree Classifier

Divide el espacio de características en regiones más pequeñas utilizando condiciones basadas en características, formando una estructura de árbol.

In [10]:
from sklearn.tree import DecisionTreeClassifier

x= datos[['edad_del_modelo','km_por_ano', 'precio']]
y= datos.vendido

SEED = 42
np.random.seed(SEED)

raw_x_train, raw_x_test, y_train, y_test = train_test_split(x,y,test_size=0.25,stratify=y)
print(f"Entrenaremos con {len(raw_x_train)} elementos y probaremos con {len(raw_x_test)} elementos.")

scaler.fit(raw_x_train)
x_train = scaler.transform(raw_x_train)
x_test = scaler.transform(raw_x_test)

model = DecisionTreeClassifier(max_depth=3)
model.fit(x_train,y_train)
previsiones= model.predict(x_test)

tasa_de_acierto = accuracy_score(y_test, previsiones)
print(f'La tasa de acierto fue de: {round(tasa_de_acierto*100,2)}%')

Entrenaremos con 7500 elementos y probaremos con 2500 elementos.
La tasa de acierto fue de: 78.04%


### Evaluación del Modelo

#### Benchmark con DummyClassifier


Un punto de referencia o "benchmark" básico para comparar con otros modelos más sofisticados. El clasificador "dummy" no intenta aprender patrones de los datos; en este caso, usa la estrategia most_frequent, que predice siempre la clase más común en el conjunto de entrenamiento.

Esto proporciona una línea base de exactitud contra la cual puedes medir el desempeño de tu modelo real. Si nuestro modelo no supera significativamente la exactitud del DummyClassifier, probablemente necesitemos revisar y mejorar tu modelo o datos.

In [11]:
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(strategy='stratified')
dummy.fit(x_train,y_train)
exactitud = dummy.score(x_test,y_test)*100
print(f'La exactitud del clasificador Dummy stratified fue: {round(exactitud,2)}%')

La exactitud del clasificador Dummy stratified fue: 52.64%


In [12]:
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(x_train,y_train)
exactitud = dummy.score(x_test,y_test)*100
print(f'La exactitud del clasificador Dummy most_frequent fue: {round(exactitud,2)}%')

La exactitud del clasificador Dummy most_frequent fue: 58.0%


### Ajuste de Hiperparametros

Seleccionamos los mejores valores para los parámetros de un modelo que no se aprenden directamente del proceso de entrenamiento.

In [13]:
from sklearn.tree import DecisionTreeClassifier

x= datos[['edad_del_modelo','km_por_ano', 'precio']]
y= datos.vendido

SEED = 42
np.random.seed(SEED)

raw_x_train, raw_x_test, y_train, y_test = train_test_split(x,y,test_size=0.25,stratify=y)
print(f"Entrenaremos con {len(raw_x_train)} elementos y probaremos con {len(raw_x_test)} elementos.")

scaler = StandardScaler()#Estandarizacion

scaler.fit(raw_x_train)
x_train = scaler.transform(raw_x_train)
x_test = scaler.transform(raw_x_test)

model = DecisionTreeClassifier(max_depth=3)
model.fit(x_train,y_train)
previsiones= model.predict(x_test)

tasa_de_acierto = accuracy_score(y_test, previsiones)
print(f'La tasa de acierto fue de: {round(tasa_de_acierto*100,2)}%')

Entrenaremos con 7500 elementos y probaremos con 2500 elementos.
La tasa de acierto fue de: 78.04%


In [14]:
from sklearn.tree import DecisionTreeClassifier

x= datos[['edad_del_modelo','km_por_ano', 'precio']]
y= datos.vendido

SEED = 42
np.random.seed(SEED)

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25,stratify=y)
print(f"Entrenaremos con {len(x_train)} elementos y probaremos con {len(x_test)} elementos.")

# Sin estandarizar

model = DecisionTreeClassifier(max_depth=3)
model.fit(x_train,y_train)
previsiones= model.predict(x_test)

tasa_de_acierto = accuracy_score(y_test, previsiones)
print(f'La tasa de acierto fue de: {round(tasa_de_acierto*100,2)}%')

Entrenaremos con 7500 elementos y probaremos con 2500 elementos.
La tasa de acierto fue de: 78.04%


In [15]:
model = DecisionTreeClassifier(max_depth=5)
model.fit(x_train,y_train)
previsiones= model.predict(x_test)

tasa_de_acierto = accuracy_score(y_test, previsiones)
print(f'La tasa de acierto fue de: {round(tasa_de_acierto*100,2)}%')

La tasa de acierto fue de: 77.32%


In [16]:
model = DecisionTreeClassifier(max_depth=10)
model.fit(x_train,y_train)
previsiones= model.predict(x_test)

tasa_de_acierto = accuracy_score(y_test, previsiones)
print(f'La tasa de acierto fue de: {round(tasa_de_acierto*100,2)}%')

La tasa de acierto fue de: 75.88%


In [17]:
model = DecisionTreeClassifier(max_depth=2)
model.fit(x_train,y_train)
previsiones= model.predict(x_test)

tasa_de_acierto = accuracy_score(y_test, previsiones)
print(f'La tasa de acierto fue de: {round(tasa_de_acierto*100,2)}%')

La tasa de acierto fue de: 75.12%


## Conclusion

El Decision Tree Classifier con una profundidad máxima de 3 ha alcanzado una tasa de acierto del 78% en la predicción de ventas de vehículos. Esto representa una mejora significativa en comparación con la línea base proporcionada por el DummyClassifier con una tasa de acierto del 58%, que solo predice la clase más frecuente.

Esta mejora indica que el modelo de árbol de decisión es capaz de capturar patrones significativos en los datos que el modelo dummy no puede, sugiriendo que las características edad_del_modelo, km_por_ano, y precio son relevantes para predecir si un vehículo será vendido.

In [18]:
import graphviz
from sklearn.tree import export_graphviz

features = x.columns
dot_data = export_graphviz(model, feature_names=features, filled=True, rounded=True, class_names=['No','Sí'])
grafico = graphviz.Source(dot_data)
print(grafico)
grafico

digraph Tree {
node [shape=box, style="filled, rounded", color="black", fontname="helvetica"] ;
edge [fontname="helvetica"] ;
0 [label="precio <= 60069.984\ngini = 0.487\nsamples = 7500\nvalue = [3150, 4350]\nclass = Sí", fillcolor="#c8e4f8"] ;
1 [label="precio <= 40083.385\ngini = 0.2\nsamples = 3290\nvalue = [370, 2920]\nclass = Sí", fillcolor="#52a9e8"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
2 [label="gini = 0.0\nsamples = 1476\nvalue = [0, 1476]\nclass = Sí", fillcolor="#399de5"] ;
1 -> 2 ;
3 [label="gini = 0.325\nsamples = 1814\nvalue = [370, 1444]\nclass = Sí", fillcolor="#6cb6ec"] ;
1 -> 3 ;
4 [label="km_por_ano <= 24124.811\ngini = 0.449\nsamples = 4210\nvalue = [2780, 1430]\nclass = No", fillcolor="#f2c29f"] ;
0 -> 4 [labeldistance=2.5, labelangle=-45, headlabel="False"] ;
5 [label="gini = 0.497\nsamples = 2629\nvalue = [1418, 1211]\nclass = No", fillcolor="#fbede2"] ;
4 -> 5 ;
6 [label="gini = 0.239\nsamples = 1581\nvalue = [1362, 219]\nclass = No", f

ExecutableNotFound: failed to execute ['dot', '-Kdot', '-Tsvg'], make sure the Graphviz executables are on your systems' PATH